In [4]:
import sys

# setting path
sys.path.append('..')

import gpflow
from gpflow.mean_functions import Constant
from gpflow.utilities import positive, print_summary
from gpflow.utilities.ops import broadcasting_elementwise
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
# from rdkit.Chem import AllChem, Descriptors, MolFromSmiles
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

from src.problems.contamination import Contamination

from src.models.GPr import GPr

In [5]:
from ngboost import NGBRegressor
from ngboost.distns import Exponential, Normal
from ngboost.scores import LogScore, CRPScore

from ngboost.distns.normal import Normal
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV


In [6]:
class Tanimoto(gpflow.kernels.Kernel):
    def __init__(self):
        super().__init__()
        # We constrain the value of the kernel variance to be positive when it's being optimised
        self.variance = gpflow.Parameter(1.0, transform=positive())

    def K(self, X, X2=None):
        """
        Compute the Tanimoto kernel matrix σ² * ((<x, y>) / (||x||^2 + ||y||^2 - <x, y>))
        :param X: N x D array
        :param X2: M x D array. If None, compute the N x N kernel matrix for X.
        :return: The kernel matrix of dimension N x M
        """
        if X2 is None:
            X2 = X

        Xs = tf.reduce_sum(tf.square(X), axis=-1)  # Squared L2-norm of X
        X2s = tf.reduce_sum(tf.square(X2), axis=-1)  # Squared L2-norm of X2
        outer_product = tf.tensordot(X, X2, [[-1], [-1]])  # outer product of the matrices X and X2

        # Analogue of denominator in Tanimoto formula

        denominator = -outer_product + broadcasting_elementwise(tf.add, Xs, X2s)

        return self.variance * outer_product/denominator

    def K_diag(self, X):
        """
        Compute the diagonal of the N x N kernel matrix of X
        :param X: N x D array
        :return: N x 1 array
        """
        return tf.fill(tf.shape(X)[:-1], tf.squeeze(self.variance))
    
def transform_data(X_train, y_train, X_test, y_test):
    """
    Apply feature scaling to the data. Return the standardised train and
    test sets together with the scaler object for the target values.
    :param X_train: input train data
    :param y_train: train labels
    :param X_test: input test data
    :param y_test: test labels
    :return: X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled, y_scaler
    """

    x_scaler = StandardScaler()
    X_train_scaled = x_scaler.fit_transform(X_train)
    X_test_scaled = x_scaler.transform(X_test)
    y_scaler = StandardScaler()
    y_train_scaled = y_scaler.fit_transform(y_train)
    y_test_scaled = y_scaler.transform(y_test)

    return X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled, y_scaler

In [7]:
#opt = LatinSquare(n=2000)
opt = Contamination(n=300, lamda=0.0001)
#opt = RNA(n=1000)
X = opt.X
y = opt.y

test_set_size = 0.2

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_set_size, random_state=0)

y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

#  We standardise the outputs but leave the inputs unchanged

_, y_train, _, y_test, y_scaler = transform_data(X_train, y_train, X_test, y_test)

X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)

k = Tanimoto()
# k = gpflow.kernels.Matern32()
m = gpflow.models.GPR(data=(X_train, y_train), mean_function=Constant(np.mean(y_train)), kernel=k, noise_variance=0.00001)


/Users/roinaveiro/miniconda/envs/tf-sbbo/lib/python3.8/site-packages/gpflow/experimental/utils.py:42: UserWarning: You're calling gpflow.experimental.check_shapes.checker.ShapeChecker.__init__ which is considered *experimental*. Expect: breaking changes, poor documentation, and bugs.
  warn(


In [9]:
opt = gpflow.optimizers.Scipy()
opt.minimize(m.training_loss, m.trainable_variables)
print_summary(m)

╒═════════════════════════╤═══════════╤══════════════════╤═════════╤═════════════╤═════════╤═════════╤══════════╕
│ name                    │ class     │ transform        │ prior   │ trainable   │ shape   │ dtype   │    value │
╞═════════════════════════╪═══════════╪══════════════════╪═════════╪═════════════╪═════════╪═════════╪══════════╡
│ GPR.mean_function.c     │ Parameter │ Identity         │         │ True        │ ()      │ float64 │ -1.99929 │
├─────────────────────────┼───────────┼──────────────────┼─────────┼─────────────┼─────────┼─────────┼──────────┤
│ GPR.kernel.variance     │ Parameter │ Softplus         │         │ True        │ ()      │ float64 │  1.38255 │
├─────────────────────────┼───────────┼──────────────────┼─────────┼─────────────┼─────────┼─────────┼──────────┤
│ GPR.likelihood.variance │ Parameter │ Softplus + Shift │         │ True        │ ()      │ float64 │  1e-05   │
╘═════════════════════════╧═══════════╧══════════════════╧═════════╧═════════════╧══════

In [10]:
y_pred, y_var = m.predict_f(X_test)
y_pred = y_scaler.inverse_transform(y_pred)
y_test = y_scaler.inverse_transform(y_test)

In [11]:
y_var.numpy().squeeze()

array([0.22587494, 0.1708074 , 0.25952579, 0.33254053, 0.14409313,
       0.1493773 , 0.13667641, 0.29051033, 0.23346279, 0.16590711,
       0.20972049, 0.23950956, 0.13632295, 0.26533296, 0.21963229,
       0.34603636, 0.27542341, 0.30040972, 0.23712465, 0.21432362,
       0.21840488, 0.25260913, 0.24233801, 0.41038211, 0.21873178,
       0.16572218, 0.41038804, 0.34170634, 0.2470014 , 0.55108049,
       0.30722127, 0.13718738, 0.26806452, 0.2363754 , 0.22523546,
       0.2920417 , 0.23386242, 0.16815049, 0.25804624, 0.17710842,
       0.26158086, 0.22834955, 0.23436548, 0.22887403, 0.19974295,
       0.28369395, 0.3178551 , 0.17353304, 0.18249934, 0.22070993,
       0.23568114, 0.29275369, 0.23351748, 0.24431235, 0.25330477,
       0.3078553 , 0.15243122, 0.17431809, 0.3058442 , 0.13559306])

In [12]:
y_pred.squeeze()

array([2.08532888, 1.58121579, 1.26577635, 1.25628659, 2.42111972,
       2.19640902, 2.11635179, 1.47317274, 2.35196782, 2.25910744,
       1.5530987 , 2.28084458, 1.8698689 , 1.79382864, 2.63140134,
       1.70618536, 1.36715512, 1.79262783, 1.71686647, 1.81982096,
       2.08328323, 1.88364934, 1.83580796, 1.60651648, 2.56643173,
       2.04126112, 1.30397476, 1.33041619, 1.36195368, 1.22789421,
       1.21974222, 2.06583614, 1.42961076, 2.54806534, 1.69784336,
       1.50389702, 2.31761608, 2.06486642, 1.45547914, 2.2624965 ,
       2.07315237, 2.21878874, 2.38627162, 2.6134526 , 2.18238926,
       0.98716891, 1.67042866, 2.07844642, 2.34199227, 1.52713945,
       1.61006416, 1.48754988, 1.6751959 , 2.31216076, 2.30277411,
       1.4064771 , 3.01780297, 2.66489568, 1.71932443, 2.55281461])

In [13]:
y_pred_train, _ = m.predict_f(X_train)
train_rmse_stan = np.sqrt(mean_squared_error(y_train, y_pred_train))
train_rmse = np.sqrt(mean_squared_error(y_scaler.inverse_transform(y_train), y_scaler.inverse_transform(y_pred_train)))
print("\nTrain RMSE (Standardised): {:.3f} nm".format(train_rmse_stan))
print("Train RMSE: {:.3f} nm".format(train_rmse))


# Output R^2, RMSE and MAE on the test set
score = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print("\nTest R^2: {:.3f}".format(score))
print("Test RMSE: {:.3f} nm".format(rmse))
print("Test MAE: {:.3f} nm".format(mae))


Train RMSE (Standardised): 0.000 nm
Train RMSE: 0.000 nm

Test R^2: 0.776
Test RMSE: 0.268 nm
Test MAE: 0.221 nm


In [16]:
from src.uncertainty_metrics import *

scorer = CVPPDiagram()
qs, Cqs = scorer.compute(y_test, y_pred, y_var, num_bins=20)
print(np.abs(qs - Cqs))

[0.         0.03596491 0.03859649 0.0745614  0.09385965 0.07982456
 0.06578947 0.08508772 0.0377193  0.09035088 0.10964912 0.1122807
 0.08157895 0.10087719 0.10350877 0.07280702 0.05877193 0.06140351
 0.01403509 0.        ]


In [19]:
scorer = AbsoluteMiscalibrationArea()
scorer.compute(y_test, y_pred, y_var, num_bins=20)

0.06951754385964909

In [14]:
ranked_confidence_list = np.argsort(y_var, axis=0).flatten()
rmse_confidence_list = np.zeros((len(y_test) ))
mae_confidence_list = np.zeros((len(y_test) ))

for k in range(len(y_test)):

    # Construct the RMSE error for each level of confidence

    conf = ranked_confidence_list[0:k+1]
    rmse = np.sqrt(mean_squared_error(y_test[conf], y_pred[conf]))
    rmse_confidence_list[k] = rmse

    # Construct the MAE error for each level of confidence

    mae = mean_absolute_error(y_test[conf], y_pred[conf])
    mae_confidence_list[k] = mae

In [15]:
rmse_confidence_list

array([0.11441461, 0.10220833, 0.09321951, 0.1146317 , 0.10630901,
       0.09855597, 0.09395783, 0.08789417, 0.12717055, 0.1289115 ,
       0.1283693 , 0.16278254, 0.15716836, 0.18146192, 0.17606891,
       0.17127728, 0.17910408, 0.18605616, 0.18453797, 0.20694798,
       0.20258442, 0.20545297, 0.20226216, 0.19814388, 0.19782797,
       0.19937011, 0.20609066, 0.2061123 , 0.20305561, 0.21402193,
       0.21180551, 0.21276202, 0.21716206, 0.22229577, 0.23046299,
       0.23964034, 0.24192108, 0.24633989, 0.24560935, 0.24348357,
       0.25130603, 0.25269561, 0.2529903 , 0.25497804, 0.25278138,
       0.26017293, 0.25824793, 0.25651985, 0.25775545, 0.25537782,
       0.26736983, 0.26484245, 0.26549744, 0.2650699 , 0.26280532,
       0.26260627, 0.26092598, 0.25875897, 0.25659667, 0.26827485])